In [103]:
import pandas as pd
import numpy as np
import random
import os
import torch
import sys
sys.path.append('../')
from modules import utils, constants

In [104]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

In [105]:
test_df = pd.read_csv('../data/test_set_basic.csv')
test_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,proteinuria,renal_biopsy_class,anti_cardioliphin_antibodies,anti_b2gp1_antibodies,lupus_anti_coagulant,c3,c4,anti_dsdna_antibody,anti_smith_antibody,label
0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,5,0,0,0,1,1,1,0,1
3,1,0,0,0,0,1,0,0,0,1,...,0,2,0,0,0,1,1,0,0,1
4,1,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,1,1,0,0,0


In [106]:
X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_test, y_test = np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

((14000, 26), (14000,))

In [218]:
dqn_model = utils.load_dqn3('../models/dqn_balanced_30000000')
test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
test_df.head()

Using stable baselines 3
Count: 2800
Count: 5600
Count: 8400
Count: 11200
Count: 14000
Testing done.....


,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,score,is_success
0,0.0,7.0,1.0,0.0,0.0,"[anti_dsdna_antibody, joint_involvement, c3, a...",0.0,0.0,1.0
1,1.0,8.0,1.0,0.0,0.0,"[anti_dsdna_antibody, joint_involvement, c3, p...",False,0.0,True
2,2.0,7.0,1.0,1.0,1.0,"[anti_dsdna_antibody, ana, pleural_effusion, a...",False,6.0,True
3,3.0,8.0,-1.0,0.0,1.0,"[anti_dsdna_antibody, joint_involvement, c3, p...",False,0.0,False
4,4.0,9.0,-1.0,1.0,0.0,"[anti_dsdna_antibody, joint_involvement, c3, p...",False,5.0,False


In [219]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

71.89999999999999

In [220]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(7.742071428571428, 0.438)

In [221]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(71.89999999999999, 69.67780047887008, 73.53663535911315)

In [222]:
test_df.y_pred.value_counts()

0.0    7843
1.0    6157
Name: y_pred, dtype: int64

In [223]:
test_df.y_actual.value_counts()

0.0    9947
1.0    4053
Name: y_actual, dtype: int64

#### Saving files

In [102]:
#test_df.to_csv(f'../test_dfs/dqn_test_df_unbalanced_6000000.csv', index=False)
#success_df.to_csv(f'../test_dfs/dqn_success_df_unbalanced_6000000.csv', index=False)